In [2]:
import numpy as np
from keras.datasets import cifar10
from keras.layers import Activation, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import copy
from cleverhans.utils_mnist import data_mnist
from PIL import Image

Using TensorFlow backend.
/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [36]:
def to_onehot(vec):
    vec = vec.flatten()
    vec_new = np.zeros((len(vec), np.max(vec) + 1))
    vec_new[np.arange(len(vec)), vec] = 1
    return vec_new

def load_data(data='mnist'):
    if data == 'mnist':
        # Get MNIST test data
        X_train, Y_train, X_test, Y_test = data_mnist()
    elif data == 'cifar10':
        # Get Cifar10 data
        (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
        Y_train = to_onehot(Y_train.flatten())
        Y_test = to_onehot(Y_test.flatten())
    else:
        raise ValueError("Dataset not compatible.")
    return X_train, Y_train, X_test, Y_test
data='cifar10'
# Load data
X_train, Y_train, X_test, Y_test = load_data(data)

# Flatten image matrices
num_train, num_rows, num_cols, num_channels = X_train.shape
num_test, _, _, _ = X_test.shape
_, num_classes = Y_train.shape

In [76]:

class DirectNoise():
    '''
    Contains functions that apply each noise transformation on any set of input images.
    '''
    def __init__(self):
        return
    
    def apply_noise(self,data,noise_type,index=None):
        def shrink(p1_shape,p2_shape,resize_shape,stack_fn):
            result = np.zeros(data.shape)
            p1 = np.zeros(p1_shape)
            p2 = np.zeros(p2_shape)
            for i in range(num_data):
                image_array = np.uint8(data[i,:].reshape(imshape)*255)
                image = Image.fromarray(image_array).resize(resize_shape, Image.ANTIALIAS)
                image = np.array(image)/255.0
                full_image = stack_fn((p1, image, p2))
                if num_channels == 1:
                    full_image = np.expand_dims(full_image,-1)
                result[i,:] = full_image
            return result
        
        def noise(f):
            random_values = f(np.random.random(data.shape) * 0.1)
            return np.clip((data + random_values), 0.0, 1.0)

        def format_background(background):
            background = np.expand_dims(background,0)
            if num_channels == 1:
                background = np.expand_dims(background,-1)
            return np.clip(data + background, 0.0, 1.0)
        
        data = copy.deepcopy(data) 
        noise_type = noise_type
        num_data, num_row_pixels,num_col_pixels,num_channels = data.shape
        if num_channels == 1:
            imshape = (num_row_pixels,num_col_pixels)
            vpadding = (4,num_col_pixels)
            hpadding = (num_row_pixels,4)
        else:
            imshape = (num_row_pixels,num_col_pixels,num_channels)
            vpadding = (4,num_col_pixels, num_channels)
            hpadding = (num_row_pixels,4, num_channels)

        if noise_type == 'none':
            return data

        elif noise_type == 'vert_shrink25':
            return shrink(vpadding,vpadding,(num_row_pixels,num_col_pixels-8),np.vstack)

        elif noise_type == 'horiz_shrink25':
            return shrink(hpadding,hpadding,(num_row_pixels-8,num_col_pixels),np.hstack)

        elif noise_type == 'both_shrink25':
            data = shrink(hpadding,hpadding,(num_row_pixels-8,num_col_pixels),np.hstack)
            return shrink(vpadding,vpadding,(num_row_pixels,num_col_pixels-8),np.vstack)

        elif noise_type == 'light_tint':
            background = 0.2 * np.ones(imshape)
            return format_background(background)

        elif noise_type == 'gradient':
            background = np.zeros(imshape)
            for i in range(num_row_pixels):
                for j in range(num_row_pixels):
                    background[i,j] = ((i+j)/54.0) * 0.4
            return format_background(background)

        elif noise_type == 'checkerboard':
            background = np.zeros(imshape)
            for i in range(num_row_pixels):
                if (i % 4 == 0) or (i % 4 == 1):
                    background[i,:] = 0.4
                    background[:,i] = 0.2
            return format_background(background)

        elif noise_type == 'pos_noise':
            return noise(lambda x: x + 0.05)

        elif noise_type == 'mid_noise':
            return noise(lambda x: x - 0.05)

        elif noise_type == 'neg_noise':
            return noise(lambda x: -1.0 * (x + 0.05))

        else:
            raise ValueError("This noise type is not currently supported.")

    def load_adversarial_examples(self,noise,data_type,index):
        images = pkl.load(open(f'../datasets/mnist_attacks/{noise}/{data_type}_features.p','rb'))
        num_features = int(os.environ['num_features'])
        images = images.reshape(images.shape[0],num_features)
        if index:
            return images[index]
        else:
            return images

In [78]:
direct_noise.apply_noise(X_train,'vert_shrink25').shape

(50000, 32, 32, 3)

In [79]:
direct_noise.apply_noise(X_train,'both_shrink25').shape

(50000, 32, 32, 3)

In [80]:
direct_noise.apply_noise(X_train,'gradient').shape

(50000, 32, 32, 3)

In [81]:
direct_noise.apply_noise(X_train,'neg_noise').shape

(50000, 32, 32, 3)